<a href="https://colab.research.google.com/github/qqq3964/NLP-/blob/main/8%EC%9E%A5_%EC%A0%95%EA%B7%9C%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 정규화를 위한 다양한 방법들
1. MinMax scaler
2. RobustScaler
3. StandardScaler

아래는 diabetes 데이터를 이용한 이진분류 예측 딥러닝이다.</br>
이를 위해 캐글에서 데이터셋을 가져왔고 뉴럴 네트워크로 구성해보았다.

In [5]:
!unzip archive.zip

Archive:  archive.zip
  inflating: diabetes.csv            


In [6]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# cuda 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cuda')

In [8]:
df = pd.read_csv('/content/diabetes.csv')
X = df[df.columns[:-1]]
y = df['Outcome']

# X는 값을 values를 이용해서 모두 바꿔주었고 y는 tensor로 바꿔줌
X = X.values
y = torch.tensor(y.values)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33)

In [9]:
ms = MinMaxScaler()
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)
# y_train ,y_test값을 (?,1) 형태로 만들어준다.
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
y_train = ms.fit_transform(y_train)
y_test = ms.fit_transform(y_test)

In [10]:
y_test.shape

(254, 1)

In [11]:
class customDataset(Dataset):
  def __init__(self,X,y):
    self.X = X
    self.y = y
    self.len = len(self.X)
  # index 번호를 통해서 객체 자체로 접근한다.
  def __getitem__(self,index):
    return self.X[index],self.y[index]
  def __len__(self):
    return self.len

In [12]:
train_data = customDataset(torch.FloatTensor(X_train),
                           torch.FloatTensor(y_train))
test_data = customDataset(torch.FloatTensor(X_test),
                           torch.FloatTensor(y_test))

train_loader = DataLoader(dataset=train_data,batch_size=64,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=64,shuffle=False)

이런식으로 맞춰줌

In [13]:
torch.FloatTensor(X_train)[1]

tensor([ 0.0185, -0.0125, -0.0507, -1.2757, -0.6773, -0.2947,  0.7320,  0.0371])

In [14]:
torch.FloatTensor(y_train)[1]

tensor([0.])

### Model 설계

In [15]:
class binaryClassifier(nn.Module):
  def __init__(self):
    super(binaryClassifier,self).__init__()
    # 칼럼이 8개이므로 처음에 들어오는 input은 8임
    self.layer1 = nn.Linear(8,64,bias=True)
    self.layer2 = nn.Linear(64,64,bias=True)
    self.layer_out = nn.Linear(64,1,bias=True)
    self.relu = nn.ReLU() # activation 함수
    self.dropout = nn.Dropout(p=0.1)
    # 정규화를 통해 빠른 연산과 gradient vanishing 현상에 대해 줄 일 수 있다.
    self.batchnorm1 = nn.BatchNorm1d(64)
    self.batchnorm2 = nn.BatchNorm1d(64)

  def forward(self,inputs):
    x1 = self.relu(self.layer1(inputs))
    x1 = self.batchnorm1(x1)
    x2 = self.relu(self.layer2(x1))
    x2 = self.batchnorm2(x2)
    x2 = self.dropout(x2)
    out = self.relu(self.layer_out(x2))
    return out

### 모델 학습

In [16]:
epochs = 1000+1
print_epoch = 100
LEARNING_RATE = 1e-2

model = binaryClassifier()
# cuda 연산을위해 model과 data를 gpu 메모리 위로 올려줘야한다.
model.to(device)
print(model)
# binary cross entropy -> loss 함수 2개의 이진분류이니 Logits 만약 다중분류면 softmax사용
BCE = nn.BCEWithLogitsLoss()
# optimizer 선언 
optimizer = torch.optim.SGD(model.parameters(),lr=LEARNING_RATE)

binaryClassifier(
  (layer1): Linear(in_features=8, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [17]:
# 모델 성능 측정 함수 정의

def accuracy(y_pred,y_test):
  # 반올림함수로 torch.round를 정의
  y_pred_tag = torch.round(torch.sigmoid(y_pred))
  correct_results_sum = (y_pred_tag == y_test).sum().float() # 실제 정답과 모델의 결과가 일치하는 개수를 실수 형태로 변수에 저장
  acc = correct_results_sum/y_test.shape[0]
  acc = torch.round(acc*100)
  return acc                          

### 모델 학습

In [18]:
for epoch in range(epochs):
  iteration_loss = 0. # 변수를 0으로 초기화
  iteration_accuracy = 0.

  # 모델학습
  model.train()
  for i,data in enumerate(train_loader):
    X,y = data
    # 모델에 넣어줄때 gpu memory 위로 같이 올려줘야한다.
    X = X.to(device)
    y = y.to(device)

    # 이제 모델에 넣고 예측
    y_pred = model(X)
    loss = BCE(y_pred,y.float())

    iteration_loss += loss # loss 값을 변수에 누적하여 저장
    iteration_accuracy += accuracy(y_pred,y.float()) # 모델 정확도 누적 저장
    
    optimizer.zero_grad() # 초기화
    loss.backward() # 데이터셋 배치사이즈로 돌았으니 backpropagation을 실행해줘야한다.
    optimizer.step()

  if(epoch % print_epoch == 0):
    # len(train_loader)로 나누는 이유는 전체 학습후에 그러면 한 iterlation 을 돌았기 때문 그래서 저런식으로 나눠준것이다. iteration 횟수만큼 누적되어 loss가 나오기떄문
    print('Train: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}'.format(epoch,iteration_loss/len(train_loader),iteration_accuracy/len(train_loader)))
  
  iteration_loss = 0. # 변수를 0으로 초기화
  iteration_accuracy = 0.
  
  # 모델 평가
  model.eval()
  for i,data in enumerate(test_loader):
    X,y = data
    # 모델에 넣어줄때 gpu memory 위로 같이 올려줘야한다.
    X = X.to(device)
    y = y.to(device)

    # 이제 모델에 넣고 예측
    y_pred = model(X)
    loss = BCE(y_pred,y.float())

    iteration_loss += loss # loss 값을 변수에 누적하여 저장
    iteration_accuracy += accuracy(y_pred,y.float()) # 모델 정확도 누적 저장

  if(epoch % print_epoch == 0):
    print('Test: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}'.format(epoch,iteration_loss/len(test_loader),iteration_accuracy/len(test_loader)))


Train: epoch: 0 - loss: 0.77103; acc: 37.556
Test: epoch: 0 - loss: 0.72878; acc: 26.500
Train: epoch: 100 - loss: 0.62363; acc: 80.667
Test: epoch: 100 - loss: 0.71492; acc: 72.500
Train: epoch: 200 - loss: 0.62884; acc: 71.222
Test: epoch: 200 - loss: 0.70390; acc: 75.750
Train: epoch: 300 - loss: 0.60356; acc: 81.333
Test: epoch: 300 - loss: 0.73325; acc: 74.750
Train: epoch: 400 - loss: 0.61287; acc: 82.333
Test: epoch: 400 - loss: 0.73294; acc: 74.250
Train: epoch: 500 - loss: 0.67980; acc: 70.889
Test: epoch: 500 - loss: 0.70963; acc: 76.750
Train: epoch: 600 - loss: 0.62406; acc: 77.222
Test: epoch: 600 - loss: 0.76122; acc: 76.250
Train: epoch: 700 - loss: 0.63093; acc: 72.000
Test: epoch: 700 - loss: 0.74302; acc: 74.750
Train: epoch: 800 - loss: 0.62209; acc: 81.889
Test: epoch: 800 - loss: 0.74624; acc: 77.500
Train: epoch: 900 - loss: 0.59166; acc: 83.667
Test: epoch: 900 - loss: 0.76293; acc: 76.500
Train: epoch: 1000 - loss: 0.60047; acc: 83.111
Test: epoch: 1000 - loss: 